# Задача
Распознать число на изображении

`картинка числа` -> преобразуется в опознанное число

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import requests
import gzip
import cv2
import zlib
import idx2numpy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [9]:
def load_ubyte(url, target_file):
    r = requests.get(url)
    data = zlib.decompress(r.content, zlib.MAX_WBITS|32)
    with open(target_file, 'wb') as f:
        f.write(data)

# Загрузим MNIST (рукописные изображения цифр)
# MNIST очень популярен для проверки качества в задачах распознания объектов
load_ubyte(
    'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
    'train_data.ubyte'
)
load_ubyte(
    'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
    'train_labels.ubyte'
)
load_ubyte(
    'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
    'test_data.ubyte'
)
load_ubyte(
    'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz',
    'test_labels.ubyte'
)

In [10]:
# Распакуем данные и считаем картинки в массив
train_images_file = 'train_data.ubyte'
train_labels_file = 'train_labels.ubyte'
test_images_file = 'test_data.ubyte'
test_labels_file = 'test_labels.ubyte'

train_images = idx2numpy.convert_from_file(train_images_file)
train_labels = idx2numpy.convert_from_file(train_labels_file)
test_images = idx2numpy.convert_from_file(test_images_file)
test_labels = idx2numpy.convert_from_file(test_labels_file)

np.set_printoptions(linewidth=300)
print(f'Одна картинка - это набор пикселей:')
print(*train_images[2][5:-3, 2:-4], sep='\n')

Одна картинка - это набор пикселей:
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  67 232  39   0]
[  0   0  62  81   0   0   0   0   0   0   0   0   0   0   0   0   0   0 120 180  39   0]
[  0   0 126 163   0   0   0   0   0   0   0   0   0   0   0   0   0   2 153 210  40   0]
[  0   0 220 163   0   0   0   0   0   0   0   0   0   0   0   0   0  27 254 162   0   0]
[  0   0 222 163   0   0   0   0   0   0   0   0   0   0   0   0   0 183 254 125   0   0]
[  0  46 245 163   0   0   0   0   0   0   0   0   0   0   0   0   0 198 254  56   0   0]
[  0 120 254 163   0   0   0   0   0   0   0   0   0   0   0   0  23 231 254  29   0   0]
[  0 159 254 120   0   0   0   0   0   0   0   0   0   0   0   0 163 254 216  16   0   0]
[  0 159 254  67   0   0   0   0   0   0   0   0   0  14  86 178 248 254  91   0   0   0]
[  0 159 254  85   0   0   0  47  49 116 144 150 241 243 234 179 241 252  40   0   0   0]
[  0 150 253 237 207 207 207 253 254 250 240 198 143  91  28   5

In [11]:
# Модели удобнее работать с векторами - распакуем массив пикселей в вектор
width, height = 28, 28
pixels = width * height
train_data = pd.DataFrame(
    train_images.reshape(-1, pixels),
    columns=[f'pixel_{i}' for i in range(pixels)]
)
test_data = pd.DataFrame(
    test_images.reshape(-1, pixels),
    columns=[f'pixel_{i}' for i in range(pixels)]
)
# Модель будет учиться по меткам, которые ей отдадут
train_data['label'] = train_labels

In [13]:
##### Обучаем модель
# Сначала приводим значения пикселей к схожему масштабу
# В блоке "Машинное обучение и приложения" мы узнаем, почему это нужно
scaler = StandardScaler()
X_std = scaler.fit_transform(train_data.values[:, 1:])
test_std = scaler.fit_transform(test_data.values)

y_train = train_data.label
# X_valid будет использован, чтобы досрочно завершить обучение модели,
# если ее качество перестанет улучшаться
X_train, X_valid, y_train, y_valid = train_test_split(
    X_std,
    y_train,
    test_size=0.1,
    random_state=42
)

# Про параметры бустинга поговорим в блоке "Машинное обучение и приложения"
param_list = [
    ("eta", 0.08), 
    ("max_depth", 6),
    ("subsample", 0.8),
    ("colsample_bytree", 0.8),
    ("objective", "multi:softmax"),
    ("eval_metric", "merror"),
    ("alpha", 8),
    ("lambda", 2),
    ("num_class", 10)
]
n_rounds = 600
# Если за последние 10 раундов модель не улучшается на валидации, прекратить обучение
early_stopping = 10
    
d_train = xgb.DMatrix(X_train, label=y_train)
d_val = xgb.DMatrix(X_valid, label=y_valid)
eval_list = [(d_train, "train"), (d_val, "validation")]
bst = xgb.train(
    param_list,
    d_train,
    n_rounds,
    evals=eval_list,
    early_stopping_rounds=early_stopping,
    verbose_eval=True
)

[0]	train-merror:0.03396	validation-merror:0.03667
[1]	train-merror:0.00800	validation-merror:0.00883
[2]	train-merror:0.00507	validation-merror:0.00583
[3]	train-merror:0.00541	validation-merror:0.00633
[4]	train-merror:0.00018	validation-merror:0.00033
[5]	train-merror:0.00000	validation-merror:0.00000
[6]	train-merror:0.00000	validation-merror:0.00000
[7]	train-merror:0.00000	validation-merror:0.00000
[8]	train-merror:0.00000	validation-merror:0.00000
[9]	train-merror:0.00000	validation-merror:0.00000
[10]	train-merror:0.00000	validation-merror:0.00000
[11]	train-merror:0.00000	validation-merror:0.00000
[12]	train-merror:0.00000	validation-merror:0.00000
[13]	train-merror:0.00000	validation-merror:0.00000
[14]	train-merror:0.00000	validation-merror:0.00000
[15]	train-merror:0.00000	validation-merror:0.00000


In [14]:
def imshow(img):
    import cv2
    import IPython
    from ipywidgets import widgets, HBox
    _,ret = cv2.imencode('.jpg', img)
    i = IPython.display.Image(data=ret, width=200, height=200)
    IPython.display.display(i)

In [15]:
# Вы можете поиграться здесь и поменять значения offset и n_show
# Например, если вместо 0, 10 написать 10, 30
# то выведутся изображения с 10-го по 40-ое и опознанные моделью числа
offset, n_show = 10, 10
for image, label in zip(
    test_data.values.reshape(-1, width, height)[offset:offset+n_show],
    test_labels[offset:offset+n_show]
):
    imshow(image)
    print(f'Опознано число: {label}')

Опознано число: 0


Опознано число: 6


Опознано число: 9


Опознано число: 0


Опознано число: 1


Опознано число: 5


Опознано число: 9


Опознано число: 7


Опознано число: 3


Опознано число: 4
